### RAA_Naver_많이본뉴스기사제목

    1. Web : https://finance.naver.com/news/news_list.nhn?mode=RANK

    2. ftn:
        a. Webpage에서 기사 제목(Title), 뉴스 URL(Href), 뉴스 매체(Press) 크롤링
 
    3. main_cr:
        a. ftn에서 얻은 뉴스 URL(Href)를 바탕으로 본문 크롤링
        b. 뉴스의 양이 너무 많아서 사용 X
        
    4. 용량문제 : 2017년도부터 적재

In [3]:
import urllib.request as req
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
from datetime import datetime

import os
import re

# os.chdir("price") #Path

In [4]:
def main_cr(href):
    mainres = req.urlopen("https://finance.naver.com/" + href)
    mainsoup = BeautifulSoup(mainres,"lxml")
    mainsoup = mainsoup.find("div",class_="articleCont")
    for i in mainsoup.find_all("div"):
        i.decompose()
    return mainsoup.text

In [5]:
def ftn(date,page):
    url = 'https://finance.naver.com/news/news_list.nhn?mode=RANK&date={}&page={}'.format(date,str(page))
    res = req.urlopen(url)
    soup = BeautifulSoup(res,"lxml")

    tmp_soup = soup.find("ul",class_="newsList").find_all("a")
    title = [x.get("title") for x in tmp_soup]
    href = [x.get("href") for x in tmp_soup]
    press = [x.text.strip() for x in soup.find_all("span",class_ = "press")]

#     main = [main_cr(x) for x in href]
#     return pd.DataFrame({"Title":title,"Href":href,"Date":date,"Press":press, "Main" : main})
    return pd.DataFrame({"Title":title,"Href":href,"Date":date,"Press":press})

    1. 해당 일자에 뉴스가 없을 때 Continue
        if len(soup.find("ul",class_="newsList").find_all("a")) == 0:
        continue
        
    2. 있으면 Lastpage 찾기
        a. Lastpage 있으면 page 돌면서 크롤링
        
        b. 없으면 첫번째 page만 크롤링

In [6]:
Total = pd.DataFrame({"Title":[],
                        "Href":[],
                        "Date":[],
                        "Press":[]})
for yy in range(2007,2022):
    for mm in range(1,13):
        for dd in range(1,32):
            date = str(yy*10000+mm*100+dd)
            res = req.urlopen("https://finance.naver.com/news/news_list.nhn?mode=RANK&date={}&page=1".format(date))
            soup = BeautifulSoup(res,"lxml")
            if len(soup.find("ul",class_="newsList").find_all("a")) == 0:
                continue
            try:
                Last_page = int(soup.find("td",class_ = "pgRR").find("a").get("href").split("=")[-1])
                date_total = pd.concat([ftn(date,page) for page in range(1,Last_page+1)]
                                  ,ignore_index=True)
            except:
                date_total = ftn(date,1)
            Total = pd.concat([Total,date_total])
            print(date)

20070101


In [13]:
Total

,Title,Href,Date,Press
0,<표> 금융기관.연구소 올해 평균환율 전망,/news/news_read.nhn?article_id=0001509760&offi...,20070101,연합뉴스
1,<작년 `매출 10조-순익1조 클럽' 가입 기업 8개>,/news/news_read.nhn?article_id=0001509854&offi...,20070101,연합뉴스
2,"삼성전자, 와이브로 상용 장비 본격 수출",/news/news_read.nhn?article_id=0001509942&offi...,20070101,연합뉴스
3,"신한은행, 새해맞이 '황금돼지페스티발' 실시",/news/news_read.nhn?article_id=0000276334&offi...,20070101,뉴시스
4,"<2007년 증시>""코스피 1740선까지 간다""",/news/news_read.nhn?article_id=0000276381&offi...,20070101,뉴시스
...,...,...,...,...
101,황금돼지해 증시 1600을 바라본다,/news/news_read.nhn?article_id=0000056657&offi...,20070101,헤럴드POP
102,“해외펀드 투자전망 여전히 밝다”,/news/news_read.nhn?article_id=0000056660&offi...,20070101,헤럴드POP
103,"중국ㆍ베트남 해외주식, 국내 대형주 투자해",/news/news_read.nhn?article_id=0000056662&offi...,20070101,헤럴드POP
104,"국내외 펀드투자, 비용절감시대",/news/news_read.nhn?article_id=0000056665&offi...,20070101,헤럴드POP


    1. Href : &mode이후는 없어도 뉴스기사 찾을 수 있음 >> 제거

In [11]:
Total["Href"] = Total.Href.map(lambda x:'https://finance.naver.com' + re.sub(r'\&mode[a-z|A-Z|0-9|=\&]*','',x))
Total['Date'] = Total.Date.astype(str)
Total = Total.rename(columns={
                'Title' : '뉴스제목명',
                'Href' : '뉴스URL',
                'Date' : '기준일자' 
})
Total[Total.기준일자.astype(int).map(lambda x:True if x >= 20170101 else False)].iloc[:,[2,1,0]].to_csv("RAA_Naver_많이본뉴스기사.csv"
                           ,encoding="utf-8-sig",index = False)